# Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
# from d2l import torch as d2l
# from torch import nn
from sklearn.model_selection import train_test_split
import csv
# import torch

In [2]:
import os

# Check if running in a conda environment
if 'CONDA_DEFAULT_ENV' in os.environ:
    print("Running in a conda environment. Conda environment:", os.environ['CONDA_DEFAULT_ENV'])
else:
    print("Not running in a conda environment.")

# Check if running in a virtual environment
if 'VIRTUAL_ENV' in os.environ:
    print("Running in a virtual environment. Virtual environment:", os.environ['VIRTUAL_ENV'])
else:
    print("Not running in a virtual environment.")

Running in a conda environment. Conda environment: ML_Labs
Not running in a virtual environment.


# Load Dataset

In [3]:
kaggle_data = pd.read_json('train.json')
kaggle_data.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [4]:
# Split the first 90% of the data as the train set
test_size = round(len(kaggle_data)*0.1) 
train_size = len(kaggle_data) - test_size

# train_set, test_set = train_test_split(kaggle_data, test_size=len(kaggle_data)-train_size, random_state = False)
train_set = kaggle_data.iloc[:train_size].copy()
test_set = kaggle_data.iloc[train_size:].copy()

# train_set
test_size

kaggle_data['full_text']

0       Design Thinking for innovation reflexion-Avril...
1       Diego Estrada\n\nDesign Thinking Assignment\n\...
2       Reporting process\n\nby Gilberto Gamboa\n\nCha...
3       Design Thinking for Innovation\n\nSindy Samaca...
4       Assignment:  Visualization Reflection  Submitt...
                              ...                        
6802    EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE    My w...
6803    Why Mind Mapping?\n\nMind maps are graphical r...
6804    Challenge\n\nSo, a few months back, I had chos...
6805    Brainstorming\n\nChallenge & Selection\n\nBrai...
6806    Mind Mapping\n\nChallenge\n\nMy consulting tea...
Name: full_text, Length: 6807, dtype: object

In [1]:
essays = kaggle_data['full_text'].tolist()
tokens = kaggle_data['tokens'].tolist()

essays

NameError: name 'kaggle_data' is not defined

# Word Embedding

In [6]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [7]:
# import spaCy's language model
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [8]:
test_set['lemmatized_text'] = lemmatization(test_set['full_text'])

In [9]:
train_set['lemmatized_text'] = lemmatization(train_set['full_text'])

In [10]:
train_set.to_csv("lem_train_set.csv", index=False)

In [20]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.load("./saved_model")

# elmo = tf.keras.layers.TFSMLayer("./saved_model", call_endpoint='serving_default')
elmo

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x1d2bf258ca0>

In [21]:
# Convert the input text into a tensor
input_text = tf.constant(['Thanks'])
trial = []

sequence_len = tf.constant([len(tokens) for tokens in kaggle_data['tokens']])

# Pass the tensor to the Elmo model
embeddings = elmo.signatures['default'](text=input_text)

KeyError: 'default'

In [ ]:
embeddings

In [ ]:
# Find longest len of tokens
max_index = kaggle_data['tokens'].apply(len).idxmax()
max_length = len(kaggle_data.loc[max_index, 'tokens'])

# Get the row with the longest list of tokens
row_with_max_tokens = kaggle_data.loc[max_index]

print("Row with the longest list of tokens:")
# print(row_with_max_tokens)
print("Length of the longest list of tokens:", max_length)



In [ ]:
# Save the model
# Define the path where you want to save the model
# saved_model_path = "./saved_model"

# # Save the model
# tf.saved_model.save(elmo, saved_model_path)

In [ ]:
elmo.Trainable = True

# Embed the whole dataset